<a href="https://colab.research.google.com/github/reyllama/TTIO/blob/master/09_Attention_BackTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
data = pd.read_csv("/content/drive/My Drive/core_stocks.csv", dtype={'stock_id': str})
data['stock_id'] = data['stock_id'].astype(str)
print(data.shape[0]//979)
df = pd.read_csv('/content/drive/My Drive/corr1.csv')
df = df.set_index(keys='stock_id')
df.columns = df.columns.astype(str)
df.index = df.index.astype(str)

1586


In [3]:
def get_embeddings(corr_df=None, n_epochs=100, emb_dim=32, lr=0.01, silent=False):
  dtype = torch.FloatTensor
  epochs = n_epochs
  embeddings = Variable(torch.randn(emb_dim, len(corr_df)).type(dtype), requires_grad=True) # Initialize with Standard Gaussian
  optimizer = optim.Adam([embeddings], lr=lr)
  mean = np.abs(corr_df).mean().mean()
  coef = 1/mean # Hyper Parameter
  loss_track = []

  for epoch in range(epochs):
    loss = 0
    for i in range(len(corr_df)):
      v1 = embeddings[:, i].view(emb_dim, -1)
      dist = torch.norm(v1 - embeddings, dim=0).view(len(corr_df), 1)
      corrs = torch.from_numpy(np.abs(coef*np.array(corr_df.iloc[:, i]))-1).view(1, len(corr_df)).type(torch.FloatTensor)
    #   corrs = torch.from_numpy(np.array(df.iloc[:, i])).view(1, len(df)).type(torch.FloatTensor) # rho 교수님 버전
      loss += torch.mm(corrs, dist)/2

    #   v1 = embeddings[:, i].view(-1, emb_dim)
    #   mm = torch.mm(v1, embeddings)
    #   mm[:, i] = torch.zeros(1) # 자기 자신과의 dot product은 0으로 만들어줌. 그렇지 않으면 각각이 다 0 벡터가 될 우려
    #   corrs = torch.from_numpy(coef*np.array(df.iloc[:, i])).view(len(df), 1).type(torch.FloatTensor)
    #   loss += torch.mm(mm, corrs)

    loss_track.append(loss.item())
    if not silent:
        if (epoch+1) % 20 == 0:
            print("{0}th epoch in process".format(epoch+1))
            print('running loss: {}'.format(loss.item()))
            print()

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for element in embeddings:
        element.clamp_(0,1)

  return embeddings

In [4]:
def data_preparer(n=1, n_epochs=100, lr=0.05, silent=False, corr=True):
    if corr:
        path = "/content/drive/My Drive/corr{0}.csv".format(n)
    if not corr:
        path = "/content/drive/My Drive/cov{0}.csv".format(n)
    df = pd.read_csv(path)
    df = df.set_index(keys='stock_id')
    df.columns = df.columns.astype(str)
    df.index = df.index.astype(str)

    if n==1:
        st, ed = 1, 121
    elif n==2:
        st, ed = 62, 186
    elif n==3:
        st, ed=124,248
    elif n==4:
        st, ed=186,310
    elif n==5:
        st, ed=248, 368
    elif n==6:
        st, ed=310, 432
    elif n==7:
        st, ed=368,489
    elif n==8:
        st, ed=432, 550
    else:
        print("Too Big A Number")
        return None

    embeddings = get_embeddings(corr_df=df, n_epochs=n_epochs, lr=lr, silent=silent)

    return df, embeddings, st, ed

In [5]:
!pip install ta
import ta

def bias(close=None, m=7):
  return close - close.rolling(m).mean()

def preprocessing(data=data, start=125, end=125+62, corr_df = None):

  data['stock_id'] = data['stock_id'].astype(str)
  data['year'] = data['date'].apply(lambda x: int(x[0:4]))
  data['month'] = data['date'].apply(lambda x: int(x[5:7]))

  data = data[(data['date_index']<=end)&(data['date_index']>=start)]
  data = data[data['stock_id'].isin(df.columns)]

  data['KLength'] = data['closing_price'] - data['opening_price']
  data['KUpperLength'] = data['highest_price'] - data[['opening_price', 'closing_price']].max(axis=1)
  data['temp'] = data['opening_price']-data['lowest_price']
  data['KLowerLength'] = data[['closing_price', 'temp']].min(axis=1)
  data['EMA'] = ta.trend.ema_indicator(close=data['closing_price'])
  data['MACD'] = ta.trend.macd(close=data['closing_price'])
  data['ROC'] = ta.momentum.roc(close=data['closing_price'])
  data['BIAS'] = bias(data['closing_price'])
  data['return'] = data['closing_price'].diff()
  data = data.drop(columns=['temp'])

  
  

  return data

  Created wheel for ta: filename=ta-0.5.25-cp36-none-any.whl size=24880 sha256=afce9f943cc92ca1bbb5f40df3a824a085130ee710e47e9f010b5c86cbe31908
  Stored in directory: /root/.cache/pip/wheels/2e/93/b7/cf649194508e53cee4145ffb949e9f26877a5a8dd12db9ed5b
Successfully built ta


In [6]:
def attention_optimize(train_data=None, valid_data=None, lr=0.02, n_epochs=20, embeddings=None, silent=False):

    tdmin = train_data['date_index'].min()
    tdmax = train_data['date_index'].max()
    vdmin = valid_data['date_index'].min()
    vdmax = valid_data['date_index'].max()
    train = train_data[train_data['date_index']>=tdmin+25]
    valid = valid_data[valid_data['date_index']>=vdmin]
    train_data = train[['stock_id', 'date_index', 'EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']].melt(id_vars=['stock_id', 'date_index'], var_name='indicator', value_name='indicator_values')
    valid_data = valid[['stock_id', 'date_index', 'EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']].melt(id_vars=['stock_id', 'date_index'], var_name='indicator', value_name='indicator_values')

    dtype = torch.FloatTensor
    weights = Variable(torch.randn(7, 32).type(dtype), requires_grad=True) # 7: num_indicators, 32: dim_embeddings
    optimizer = optim.Adam([weights], lr=lr)
    base_loss = np.Inf

    retur = np.array(train[train['date_index']>=tdmin+26].pivot(index='date_index', columns='stock_id', values='return')) # (days-1) x 1586
    v_retur = np.array(valid[valid['date_index']>=vdmin+1].pivot(index='date_index', columns='stock_id', values='return')) # (days-1) x 1586
    final_factor = None

    for epoch in range(n_epochs):

        weighted_indicator = None
        v_weighted_indicator = None

        # C = torch.mm(weights, embeddings) # 7 x 1586, dot-product attention
        C = torch.mm(weights, embeddings)/torch.sqrt(torch.FloatTensor([32])) # 7 x 1586, scaled dot-product attention
        C = F.softmax(C, dim=0)           # 7 x 1586, columnwise softmax

        for day in range(tdmin+25, tdmax):
            indic = np.array(train_data[train_data['date_index']==day].pivot(index='indicator', columns='stock_id', values='indicator_values')) # 7 x 1586
            rescaled_I = torch.sum(C*torch.from_numpy(indic), axis=0).view(1, 1586) # 1 x 1586, elementwise
            if weighted_indicator == None:
                weighted_indicator = rescaled_I
            else:
                weighted_indicator = torch.cat((weighted_indicator, rescaled_I), dim=0)
        vx = torch.from_numpy(retur) - torch.mean(torch.from_numpy(retur), axis=0) 
        vy = weighted_indicator - torch.mean(weighted_indicator, axis=0) 
        # loss = -torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2))) # 1st Loss
        # loss = -torch.abs(torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))) # 2nd Loss
        loss = -torch.sum(vx * vy) / 1586 # 3rd loss, covariance-like loss

        if not silent:
            if (epoch+1) % 100 == 0:
                print("{0}th epoch in process, loss: {1:.2f}".format(epoch+1, loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():

            for day in range(vdmin, vdmax):
                indic = np.array(valid_data[valid_data['date_index']==day].pivot(index='indicator', columns='stock_id', values='indicator_values')) # 7 x 1586
                rescaled_I = torch.sum(C.detach()*torch.from_numpy(indic), axis=0).view(1, 1586) # 1 x 1586
                if v_weighted_indicator == None:
                    v_weighted_indicator = rescaled_I
                else:
                    v_weighted_indicator = torch.cat((v_weighted_indicator, rescaled_I), dim=0)     

            ux = torch.from_numpy(v_retur) - torch.mean(torch.from_numpy(v_retur), axis=0)
            uy = v_weighted_indicator - torch.mean(v_weighted_indicator, axis=0)
            v_loss = (-torch.abs(torch.sum(ux * uy) / (torch.sqrt(torch.sum(ux ** 2)) * torch.sqrt(torch.sum(uy ** 2))))).detach().numpy()

            if v_loss < base_loss:
                base_loss = v_loss
                final_factor = C

    if final_factor == None:
        print('Something is wrong')
    return final_factor

In [7]:
def attention_topK(data=None, corr_df=None, start=None, end=None, window=62, embeddings=None, e=20):
    from collections import defaultdict

    first_term = preprocessing(data=data, corr_df=corr_df, start=start, end=end+2*window)
    train_df = first_term[(first_term['date_index']>=start) & (first_term['date_index']<=end)]
    valid_df = first_term[(first_term['date_index']>=end+1) & (first_term['date_index']<=end+window)]

    indicators=['EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']
    result = defaultdict(list)

    C = attention_optimize(train_data=train_df, valid_data=valid_df, n_epochs=30, embeddings=embeddings, silent=True).detach().numpy() # 7 x 1586 (num_indicators x num_stocks)

    for k in range(3, 26): # Selecting Top K

        for indicator in indicators: # Out of 7 Indicators

            j = 0 # To average the results

            for _ in range(e): # Number of backtests to conduct, averaging out later

                base_cap = 1e7 # Seed Capital
                a = [1e7] # Trajectory of capital possession

                for day in range(end+window, end+2*window):

                    base_top10 = np.array(first_term[first_term['date_index']==day][indicator]).T.argsort()[-k:][::-1]
                    base_top10_close = first_term[first_term['date_index']==day+1].iloc[base_top10, :]['closing_price']
                    base_top10_open = first_term[first_term['date_index']==day+1].iloc[base_top10, :]['opening_price']
                    base_cap += ((base_top10_close*0.997 - base_top10_open)/base_top10_open*base_cap/k).sum()
                    a.append(base_cap)


                j += a[-1]

            result['start'].append(start)
            result['end'].append(end)
            result['k'].append(k)
            result['indicator'].append(indicator)
            result['output'].append(j/e)


        sor_first_term = first_term.sort_values(by=['date_index', 'stock_id'])
        indicator_matrix = sor_first_term[(sor_first_term['date_index']>=end+1) & (sor_first_term['date_index']<end+window)][indicators].T # 7 x 1586*days
        daily_indicator = np.array((np.tile(C, window-1) * indicator_matrix).sum()).reshape(-1, 1586)

        s = 0

        for _ in range(e):

            cap = 1e7
            b = [1e7]

            for i, day in enumerate(list(range(end+1, end+window))):
                topk = daily_indicator[i, ].argsort()[-k:][::-1]
                top10_close = first_term[first_term['date_index']==day+1].iloc[topk, :]['closing_price']
                top10_open = first_term[first_term['date_index']==day+1].iloc[topk, :]['opening_price']
                cap += ((top10_close*0.997 - top10_open)/top10_open*cap/k).sum()
                b.append(cap)

            s += b[-1]

        result['start'].append(start)
        result['end'].append(end)
        result['k'].append(k)
        result['indicator'].append("Weighted")
        result['output'].append(s/e)


    return result

In [10]:
def attention_backtest(data=None, corr_df=None, start=None, end=None, window=62, embeddings=None, e=5, k=8):

    from collections import defaultdict

    first_term = preprocessing(data=data, corr_df=corr_df, start=start, end=end+2*window)
    train_df = first_term[(first_term['date_index']>=start) & (first_term['date_index']<=end)]
    valid_df = first_term[(first_term['date_index']>=end+1) & (first_term['date_index']<=end+window)]
    indicators=['EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']
    result = defaultdict(list)

    for indicator in indicators: # Out of 7 Indicators

        j = 0 # To average the results

        for _ in range(e): # Number of backtests to conduct, averaging out later

            base_cap = 1e7 # Seed Capital
            a = [1e7] # Trajectory of capital possession

            for day in range(end+window, end+2*window):

                base_top10 = np.array(first_term[first_term['date_index']==day][indicator]).T.argsort()[-k:][::-1]
                base_top10_close = first_term[first_term['date_index']==day+1].iloc[base_top10, :]['closing_price']
                base_top10_open = first_term[first_term['date_index']==day+1].iloc[base_top10, :]['opening_price']
                base_cap += ((base_top10_close*0.997 - base_top10_open)/base_top10_open*base_cap/k).sum()
                a.append(base_cap)


            j += a[-1]

        result['start'].append(start)
        result['end'].append(end)
        result['indicator'].append(indicator)
        result['output'].append(j/e)
        result['k'].append(k)
        # result['scaled'].append(s/e)
    
    C = attention_optimize(train_data=train_df, valid_data=valid_df, n_epochs=30, embeddings=embeddings, silent=True).detach().numpy() # 7 x 1586 (num_indicators x num_stocks)
    sor_first_term = first_term.sort_values(by=['date_index', 'stock_id'])
    indicator_matrix = sor_first_term[(sor_first_term['date_index']>=end+window) & (sor_first_term['date_index']<end+2*window)][indicators].T # 7 x 1586*days
    daily_indicator = np.array((np.tile(C, window) * indicator_matrix).sum()).reshape(-1, 1586)

    s = 0

    for _ in range(e):

        cap = 1e7
        b = [1e7]

        for i, day in enumerate(list(range(end+window, end+2*window))):
            topk = daily_indicator[i, ].argsort()[-k:][::-1]
            top10_close = first_term[first_term['date_index']==day+1].iloc[topk, :]['closing_price']
            top10_open = first_term[first_term['date_index']==day+1].iloc[topk, :]['opening_price']
            cap += ((top10_close*0.997 - top10_open)/top10_open*cap/k).sum()
            b.append(cap)

        s += b[-1]

        result['start'].append(start)
        result['end'].append(end)
        result['indicator'].append("Weighted")
        result['output'].append(s/e)
        result['k'].append(k)


    return result

In [ ]:
# Execute Top K selection on Validation Set

# topk_output = pd.DataFrame(columns=['start', 'end', 'k', 'indicator', 'output'])
topk_output = pd.read_csv('/content/drive/My Drive/attention_topK_validation_3_25_20_epochs_real.csv')
# period 8 continued
for n_ in range(8,9):
    df, embeddings, st, ed = data_preparer(n=n_, silent=True, n_epochs=100)
    output_dict = attention_topK(data=data, corr_df=df, start=st, end=ed, embeddings=embeddings, e=20)
    topk_output = pd.concat([topk_output, pd.DataFrame(output_dict)])
    print("Epoch {} Complete".format(n_))

topk_output.to_csv('attention_topK_validation_3_25_20_epochs_FULL.csv', index=False)
topk_output

Epoch 8 Complete


,start,end,k,indicator,output
0,1,121,3,EMA,9.615434e+06
1,1,121,3,MACD,7.746237e+06
2,1,121,3,KLength,7.627000e+06
3,1,121,3,KUpperLength,7.456177e+06
4,1,121,3,KLowerLength,9.021456e+06
...,...,...,...,...,...
179,432,550,25,KUpperLength,7.323026e+06
180,432,550,25,KLowerLength,7.298865e+06
181,432,550,25,ROC,5.565466e+06
182,432,550,25,BIAS,6.886126e+06


In [ ]:
topk_output.to_csv('/content/drive/My Drive/attention_topK_validation_3_25_20_epochs_real.csv', index=False)
topk_output

,start,end,k,indicator,output
0,1,121,3,EMA,9.615434e+06
1,1,121,3,MACD,7.746237e+06
2,1,121,3,KLength,7.627000e+06
3,1,121,3,KUpperLength,7.456177e+06
4,1,121,3,KLowerLength,9.021456e+06
...,...,...,...,...,...
179,368,489,25,KUpperLength,7.707401e+06
180,368,489,25,KLowerLength,8.536690e+06
181,368,489,25,ROC,6.612173e+06
182,368,489,25,BIAS,6.428494e+06


In [12]:
# Actual Backtest on Test Set

import time

backtest_output = pd.DataFrame(columns=['start', 'end', 'k', 'indicator', 'output'])

for n_ in range(1, 9):

    if n_==1:
        t0 = time.time()

    df, embeddings, st, ed = data_preparer(n=n_, silent=True, n_epochs=100)

    if n_==1:
        t1 = time.time()
        print("Embedding 획득에 소요된 시간: {:.1f}분".format((t1-t0)/60))
        t2 =time.time()

    for k_ in [23, 24, 22, 25, 21]:

        result_dict = attention_backtest(data=data, corr_df=df, start=st, end=ed, embeddings=embeddings, e=1, k=k_)
        backtest_output = pd.concat([backtest_output, pd.DataFrame(result_dict)])

    if n_==1:
        t3 = time.time()
        print("한 Period 도는데 걸리는 시간: {:.1f}분".format((t3-t2)/60))
    
    print("Epoch {} Complete".format(n_))

backtest_output.to_csv("/content/drive/My Drive/Attention_Backtest_k_23_24_22_25_21.csv", index=False)
print("Saved!")

backtest_output

Embedding 획득에 소요된 시간: 3.0분
한 Period 도는데 걸리는 시간: 4.0분
Epoch 1 Complete
Epoch 2 Complete
Epoch 3 Complete
Epoch 4 Complete
Epoch 5 Complete
Epoch 6 Complete
Epoch 7 Complete
Epoch 8 Complete
Saved!


,start,end,k,indicator,output
0,1,121,23,EMA,7.776338e+06
1,1,121,23,MACD,7.419183e+06
2,1,121,23,KLength,7.346115e+06
3,1,121,23,KUpperLength,7.123484e+06
4,1,121,23,KLowerLength,7.519974e+06
...,...,...,...,...,...
3,432,550,21,KUpperLength,7.266042e+06
4,432,550,21,KLowerLength,7.160804e+06
5,432,550,21,ROC,5.334275e+06
6,432,550,21,BIAS,6.824341e+06


In [19]:
t = pd.DataFrame(backtest_output.groupby(['k', 'indicator'])['output'].mean()).reset_index().pivot(index='k', columns='indicator', values='output')
t['maximum'] = t.drop(columns=['Weighted']).max(axis=1)
t['diff'] = t.Weighted - t.maximum
t.sort_values(by='diff', ascending=False).to_csv('/content/drive/My Drive/Attention_Backtest_k_23_24_22_25_21_pivot.csv', index=False)

In [20]:
t.sort_values(by='diff', ascending=False)

indicator,BIAS,EMA,KLength,KLowerLength,KUpperLength,MACD,ROC,Weighted,maximum,diff
k,,,,,,,,,,
24,7.191609e+06,8.149864e+06,7.337540e+06,8.110195e+06,8.120950e+06,7.844128e+06,5.772344e+06,8.189400e+06,8.149864e+06,39536.078202
23,7.148686e+06,8.132747e+06,7.343637e+06,8.067382e+06,8.141312e+06,7.824942e+06,5.796739e+06,8.160825e+06,8.141312e+06,19512.594008
25,7.222204e+06,8.149548e+06,7.380849e+06,8.137331e+06,8.122260e+06,7.816352e+06,5.895823e+06,8.159930e+06,8.149548e+06,10381.434214
21,7.068975e+06,8.156886e+06,7.301871e+06,8.070987e+06,8.072886e+06,7.843657e+06,5.679396e+06,8.117835e+06,8.156886e+06,-39051.194702
22,7.120944e+06,8.171936e+06,7.336365e+06,8.059467e+06,8.105514e+06,7.833318e+06,5.708791e+06,8.126519e+06,8.171936e+06,-45417.673022


In [ ]:
backtest_output.pivot(index='start', columns='indicator', values='output').to_csv('backtest_scaled_attention_covariance_loss_top10.csv')

In [ ]:
pd.DataFrame(backtest_output.groupby('indicator')['output'].mean())

,output
indicator,
BIAS,6.801616e+06
EMA,8.442260e+06
KLength,7.209335e+06
KLowerLength,8.140336e+06
KUpperLength,8.047801e+06
MACD,8.024281e+06
ROC,5.843367e+06
Weighted,8.397495e+06


In [ ]:
1+1


2